## NC Files Data Extraction 

In [39]:
#filename='PACE_OCI.20240401T123447.L2.OC_AOP.V2_0.NRT.nc'
nc_file = nc.Dataset(filename, 'r')  # 'r' stands for read-only mode

# Access a specific group within the file
group_name = 'geophysical_data'
group = nc_file[group_name]

## Access variables within the group
Rrs_data = group.variables['Rrs'][:]  # Rrs data in geophysical_data group 184bands
Rrs_unc_data = group.variables['Rrs'][:]  # Rrs_unc data in geophysical_data group 184bands
angstrom_data = group.variables['angstrom'][:] ## Angstrom data 
aot865_data=group.variables['aot_865'][:] ## Aot_865 data
l2_flag_data=group.variables['l2_flags'][:] ## l2 flag



# Access a specific group within the file
group_name2 = 'sensor_band_parameters'
group2 = nc_file[group_name2]
## Access variables within the group
Wavelength_3d_data = group2.variables['wavelength_3d'][:]  # Wavelength_3d in sensor_band_parameters
Wavelength_1d_data = group2.variables['wavelength'][:]  # Wavelength_3d in sensor_band_parameters
F0_data = group2.variables['F0'][:]   # F0 in sensor_band_parameters
Tau_r_data = group2.variables['Tau_r'][:]   # Tau_r in sensor_band_parameters
Koz_data= group2.variables['k_oz'][:]   # K_oz in sensor_band_parameters



# Access a specific group within the file
group_name3 = 'navigation_data'
group3 = nc_file[group_name3]
LO_data=group3.variables['longitude'][:] ## LO in navigation_data
LA_data=group3.variables['latitude'][:] ## LA in navigation data

# Access a specific group within the file
group_name4 = 'scan_line_attributes'
group4 = nc_file[group_name4]
csol_z_data=group4.variables['csol_z'][:] ## LO in navigation_data

# close .nc
nc_file.close()

In [11]:
## Numpy array conversion
Rrs_data=np.array(Rrs_data)
Rrs_unc_data=np.array(Rrs_unc_data)
angstrom_data=np.array(angstrom_data)
aot865_data=np.array(aot865_data)
l2_flag_data=np.array(l2_flag_data)
Wavelength_3d_data=np.array(Wavelength_3d_data)
F0_data=np.array(F0_data)
LO_data=np.array(LO_data)
LA_data=np.array(LA_data)
Tau_r_data=np.array(Tau_r_data)
Koz_data=np.array(Koz_data)
csol_z_data=np.array(csol_z_data)


## Rrs Part

In [13]:
lat=41;lon=1.36 ## Casablanca Coordinate

L=np.abs((LA_data-lat))+np.abs((LO_data-lon))

L.shape

(1709, 1272)

In [14]:
p1, q1 = np.unravel_index(np.argmin(L), L.shape) ## Indices Fiding 


l=4;                         # selecting subsampling size        
CA=LA_data[p1-l:p1+l,q1-l:q1+l]  # subsampling latitude
CO=LO_data[p1-l:p1+l,q1-l:q1+l]  # subsampling longitude


a, b = CA.shape
dist = np.zeros_like(CA)


# Assuming CA, CO, lat, and lon are NumPy arrays or matrices
# Also assuming haversine function is defined
# Initialize dist as a matrix of zeros with the same size as CA
CA_rad = np.radians(CA)
CO_rad = np.radians(CO)
lat_rad = radians(lat)
lon_rad = radians(lon)


for o in range(a):  # Loop over rows
    for rr in range(b):  # Loop over columns
        # Calculate Haversine distance between the pixel and the target location
        distv = haversine_distances([[CA_rad[o, rr], CO_rad[o, rr]]], [[lat_rad, lon_rad]])[0, 0]
        
        # Store the distance in the corresponding position in the dist matrix
        dist[o, rr] = distv   # To convert to KM (* 6371000/1000 )



In [15]:
p1,q1

(1133, 858)

In [16]:
M = np.min(dist)

# Find the indices of the minimum distance in the flattened dist matrix
I_flat = np.argmin(dist)

# Convert the flattened index to row and column indices
I_row, I_col = np.unravel_index(I_flat, dist.shape)

# Calculate the row and column numbers of the closest pixel
r = p1 - l + I_row
c = q1 - l + I_col

In [17]:
r,c

(1133, 858)

In [18]:
# Test the pixel distance from the ideal location 
LA=np.radians(LA_data)
LO=np.radians(LO_data)
pixel_distance = haversine_distances([[LA[r, c], LO[r, c]]], [[lat_rad, lon_rad]])[0, 0]
pixel_distance_KM=pixel_distance * 6371000/1000 
print('Pixel Distance:',pixel_distance_KM)


Pixel Distance: 0.5206072789323302


In [19]:
## 1.2Km 1x1 grid
Rrs_0=Rrs_data[r,c,:]
L2_flag_0=l2_flag_data[r,c]
aot865_0=aot865_data[r,c]
angstrom_0=angstrom_data[r,c]


In [20]:
# 3.6Km 3x3 grid
## Question for Eder: why calculate the mean and std
px=1
Rrs_1=Rrs_data[r-px:r+px+1,c-px:c+px+1,:]
L2_flag_1=l2_flag_data[r-px:r+px,c-px:c+px]
aot865_1=aot865_data[r-px:r+px,c-px:c+px]
#angstrom_1=angstrom_data[r-px,c+px]
## Question for Eder

In [21]:
# 6Km 5x5 grid
## Question for Eder: why calculate the mean and std
px=2
Rrs_2=Rrs_data[r-px:r+px+1,c-px:c+px+1,:]
L2_flag_2=l2_flag_data[r-px:r+px,c-px:c+px]
aot865_2=aot865_data[r-px:r+px,c-px:c+px]
#angstrom_2=angstrom_data[r-px,c+px]
## Question for Eder

In [22]:
# 8.4Km 7x7 grid
## Question for Eder: why calculate the mean and std
px=4
Rrs_3=Rrs_data[r-px:r+px+1,c-px:c+px+1,:]
L2_flag_3=l2_flag_data[r-px:r+px,c-px:c+px]
aot865_3=aot865_data[r-px:r+px,c-px:c+px]
#angstrom_3=angstrom_data[r-px,c+px]


Replace Nan before plotting

In [ ]:
## If using Grid data, replace Nan first 
Rrs_1[Rrs_1 < 0 ] = np.nan

mean_rrs = np.nanmean(Rrs_1, axis=(0, 1))

<ipython-input-23-e0d9fee91629>:4: RuntimeWarning: Mean of empty slice
  mean_rrs = np.nanmean(Rrs_1, axis=(0, 1))


Below are ARONET DATA extraction and Matching up with PACE mission Data 

In [26]:
## Locate Wavelength in .nc file variables

target_lambda_headers = np.array([413, 442, 490, 510, 560, 620, 667, 681, 709])
# Find indices of target headers
indices_lambda = np.where(Wavelength_1d_data == np.array(target_lambda_headers)[:, None])[1]


In [27]:
## Mean Solar Flux [W.m^-2 um^-1]
F0_specificed=F0_data[indices_lambda]

In [28]:

plot_indices=np.where(Wavelength_3d_data == np.array(target_lambda_headers)[:, None])[1]


## Interp Values on 3d wavelength data range

In [29]:
from scipy.interpolate import interp1d

interp_func_1 = interp1d(Wavelength_1d_data, Tau_r_data, kind='linear', fill_value="extrapolate")
# Interpolate the values at the points in wavelength_3d data
Tau_r_data_lambda3D_data = interp_func_1(Wavelength_3d_data)

tauC = np.zeros_like(Tau_r_data_lambda3D_data)  

for i in range(184):
    if i < 62:
        tauC[i] = Tau_r_data_lambda3D_data[i] - 0.045 * Tau_r_data_lambda3D_data[i] * (490 - Wavelength_3d_data[i]) / (490 - 339)
    else:
        tauC[i] = Wavelength_3d_data[i]

# Transpose tauC to match MATLAB's behavior (if needed)
tauR = tauC  

In [30]:
interp_func_2 = interp1d(Wavelength_1d_data, Koz_data, kind='linear', fill_value="extrapolate")
# Interpolate the values at the points in wavelength_3d data
Tau_oz_data_3d = interp_func_2(Wavelength_3d_data) * 0.3

interp_func_3 = interp1d(Wavelength_1d_data, F0_data/10, kind='linear', fill_value="extrapolate")
# Interpolate the values at the points in wavelength_3d data
F00_data = interp_func_3(Wavelength_3d_data) 

## csol & Solzo data interp

In [31]:
## use r,c coordinate to performe interp
csol_z_data.shape

SolZ= csol_z_data[c]

SenZ = np.degrees(np.arctan(np.abs(1272/2 - r) * 1.2 / 676.5))

## r,c coordinate aot interp
aot865_intrp = aot865_0 * (865.0 / Wavelength_3d_data) ** angstrom_0


In [32]:
Lr = F00_data * tauR * 0.75 * (1 + (np.cos(np.radians(SolZ))**2) * (np.cos(np.radians(SenZ))**2)) / (4 * np.pi * np.cos(np.radians(SenZ)))

# Compute Lr1
Lr1 = F00_data * tauR * 0.75 * (1 + (np.cos(np.radians(SolZ))**2) * (np.cos(np.radians(SenZ))**2)) / (4 * np.pi * np.cos(np.radians(SenZ)))

# Compute t0 (sun to surface)
t0 = np.exp(-((tauR / 2 + Tau_oz_data_3d) / np.cos(np.radians(SolZ))))

# Compute Ed
Ed = F00_data * t0 * np.cos(np.radians(SolZ))

# Compute La
La = F00_data * aot865_intrp * 0.15 / (4 * np.pi * np.cos(np.radians(SenZ)))

# Compute t (surface to TOA)
t = np.exp(-((tauR / 2 + Tau_oz_data_3d) / np.cos(np.radians(SenZ))))

In [33]:
# Compute S1
S1 = (Lr1 - Lr) / Ed / t

# Compute S
S = Lr / Ed

# Define wl2 array
wl2 = np.array([400, 412, 443, 490, 510, 560, 620, 667])

In [34]:
# Assuming Rrs, rV, and cV are already defined as arrays
RrsL0 = Rrs_data[r, c,:]

# Compute RrsL
RrsL = Rrs_data[r, c,:] + S1


In [35]:
X1 = np.abs(Rrs_data[r, c, 43] / Rrs_data[r, c, 86])
X2 = np.abs(Rrs_data[r, c, 60] / Rrs_data[r, c, 86])

# Calculate the maximum of X1 and X2 element-wise
X = np.log10(np.maximum(X1, X2))

In [36]:
# Coefficients
a0 = 0.2228
a1 = -2.4683
a2 = 1.5867
a3 = -0.4275
a4 = -0.7768

# Assuming X is already defined as a NumPy array
chlV2 = 10 ** (a0 + a1 * X + a2 * X**2 + a3 * X**3 + a4 * X**4)
